# Experimental: SVC, KNN etc..

In [53]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.classify import MaxentClassifier
from nltk.stem import porter
from collections import *
from sklearn.metrics import confusion_matrix
import math
import numpy as np

In [54]:
# sub-functions for find overlapping n-grams
def intersect_modified (list1, list2) :
    cnt1 = Counter()
    cnt2 = Counter()
    for tk1 in list1:
        cnt1[tk1] += 1
    for tk2 in list2:
        cnt2[tk2] += 1    
    inter = cnt1 & cnt2
    union = cnt1 | cnt2
    largeinter = Counter()
    for (element, count) in inter.items():
        largeinter[element] = union[element]
    return list(largeinter.elements())

def intersect (list1, list2) :
    cnt1 = Counter()
    cnt2 = Counter()
    for tk1 in list1:
        cnt1[tk1] += 1
    for tk2 in list2:
        cnt2[tk2] += 1    
    inter = cnt1 & cnt2
    return list(inter.elements())

def paraphrase_Das_features(source, target):
    source_words = word_tokenize(source)
    target_words = word_tokenize(target)

    features = {}

    ###### Word Features ########

    s1grams = [w.lower() for w in source_words]
    t1grams = [w.lower() for w in target_words]
    s2grams = []
    t2grams = []
    s3grams = []
    t3grams = []

    for i in range(0, len(s1grams)-1) :
        if i < len(s1grams) - 1:
            s2gram = s1grams[i] + " " + s1grams[i+1]
            s2grams.append(s2gram)
        if i < len(s1grams)-2:
            s3gram = s1grams[i] + " " + s1grams[i+1] + " " + s1grams[i+2]
            s3grams.append(s3gram)

    for i in range(0, len(t1grams)-1) :
        if i < len(t1grams) - 1:
            t2gram = t1grams[i] + " " + t1grams[i+1]
            t2grams.append(t2gram)
        if i < len(t1grams)-2:
            t3gram = t1grams[i] + " " + t1grams[i+1] + " " + t1grams[i+2]
            t3grams.append(t3gram)

    f1gram = 0        
    precision1gram = len(set(intersect(s1grams, t1grams))) / len(set(s1grams))
    recall1gram    = len(set(intersect(s1grams, t1grams))) / len(set(t1grams))
    if (precision1gram + recall1gram) > 0:
        f1gram = 2 * precision1gram * recall1gram / (precision1gram + recall1gram)
    precision2gram = len(set(intersect(s2grams, t2grams))) / len(set(s2grams))
    recall2gram    = len(set(intersect(s2grams, t2grams))) / len(set(t2grams))
    f2gram = 0
    if (precision2gram + recall2gram) > 0:
        f2gram = 2 * precision1gram * recall2gram / (precision2gram + recall2gram)
    precision3gram = len(set(intersect(s3grams, t3grams))) / len(set(s3grams))
    recall3gram    = len(set(intersect(s3grams, t3grams))) / len(set(t3grams))
    f3gram = 0
    if (precision3gram + recall3gram) > 0:
        f3gram = 2 * precision3gram * recall3gram /(precision3gram + recall3gram)

    features["precision1gram"] = precision1gram
    features["recall1gram"] = recall1gram
    features["f1gram"] = f1gram
    features["precision2gram"] = precision2gram
    features["recall2gram"] = recall2gram
    features["f2gram"] = f2gram
    features["precision3gram"] = precision3gram
    features["recall3gram"] = recall3gram
    features["f3gram"] = f3gram

    ###### Stemmed Word Features ########

    porterstemmer = porter.PorterStemmer()
    s1stems = [porterstemmer.stem(w.lower()) for w in source_words]
    t1stems = [porterstemmer.stem(w.lower()) for w in target_words]
    s2stems = []
    t2stems = []
    s3stems = []
    t3stems = []

    for i in range(0, len(s1stems)-1) :
        if i < len(s1stems) - 1:
            s2stem = s1stems[i] + " " + s1stems[i+1]
            s2stems.append(s2stem)
        if i < len(s1stems)-2:
            s3stem = s1stems[i] + " " + s1stems[i+1] + " " + s1stems[i+2]
            s3stems.append(s3stem)

    for i in range(0, len(t1stems)-1) :
        if i < len(t1stems) - 1:
            t2stem = t1stems[i] + " " + t1stems[i+1]
            t2stems.append(t2stem)
        if i < len(t1stems)-2:
            t3stem = t1stems[i] + " " + t1stems[i+1] + " " + t1stems[i+2]
            t3stems.append(t3stem)

    precision1stem = len(set(intersect(s1stems, t1stems))) / len(set(s1stems))
    recall1stem    = len(set(intersect(s1stems, t1stems))) / len(set(t1stems))
    f1stem = 0
    if (precision1stem + recall1stem) > 0:
        f1stem = 2 * precision1stem * recall1stem / (precision1stem + recall1stem)
    precision2stem = len(set(intersect(s2stems, t2stems))) / len(set(s2stems))
    recall2stem    = len(set(intersect(s2stems, t2stems))) / len(set(t2stems))
    f2stem = 0
    if (precision2stem + recall2stem) > 0:
        f2stem = 2 * precision2stem * recall2stem / (precision2stem + recall2stem)
    precision3stem = len(set(intersect(s3stems, t3stems))) / len(set(s3stems))
    recall3stem    = len(set(intersect(s3stems, t3stems))) / len(set(t3stems))
    f3stem = 0
    if (precision3stem + recall3stem) > 0:
        f3stem = 2 * precision3stem * recall3stem / (precision3stem + recall3stem)

    features["precision1stem"] = precision1stem
    features["recall1stem"] = recall1stem
    features["f1stem"] = f1stem
    features["precision2stem"] = precision2stem
    features["recall2stem"] = recall2stem
    features["f2stem"] = f2stem
    features["precision3stem"] = precision3stem
    features["recall3stem"] = recall3stem
    features["f3stem"] = f3stem

    return features


In [55]:
def get_gt_label(x):
    if int(x) in [0,1,2]:
        return 0.0
    elif int(x) in [4,5]:
        return 1.0
    else:
        return -1.0
    
def get_pred_label(x, thresh=0.5):
    return 1.0 if x>=thresh else 0.0

def get_gt_train_label(x):
    if int(x[0])==2 and int(x[1])==3:
        return -1.0
    elif int(x[0])>=int(x[1]):
        return 1.0
    else:
        return 0.0

In [68]:
dev_dp = 'SemEval-PIT2015-github/data/dev.data'
train_dp = 'SemEval-PIT2015-github/data/train.data'
test_dp = 'SemEval-PIT2015-github/data/test.data'

In [69]:
# read train data
from tqdm import tqdm 
train_examples = []
train_gt = []
sent1_train, sent2_train = [], []
train_features = []

with open(train_dp, 'r') as f:
    lines = f.readlines() #[choose:choose+1]
    for line in tqdm(lines, total=len(lines)):
        comp = line.split('\t')
        gt = get_gt_train_label(comp[4][1:-1].split(','))
        # to skip ambiguous cases..
        if gt == -1:
            continue
        sent1_train.append(comp[2])
        sent2_train.append(comp[3])
        
        feature = paraphrase_Das_features(comp[2], comp[3])
        train_features.append(feature)
        
        train_gt.append(gt)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13063/13063 [00:08<00:00, 1535.05it/s]


In [70]:
import pandas as pd
df = pd.DataFrame.from_records(train_features)
df

,precision1gram,recall1gram,f1gram,precision2gram,recall2gram,f2gram,precision3gram,recall3gram,f3gram,precision1stem,recall1stem,f1stem,precision2stem,recall2stem,f2stem,precision3stem,recall3stem,f3stem
0,0.500000,0.416667,0.454545,0.333333,0.250000,0.428571,0.125,0.090909,0.105263,0.500000,0.416667,0.454545,0.333333,0.250000,0.285714,0.125,0.090909,0.105263
1,0.700000,0.636364,0.666667,0.333333,0.272727,0.630000,0.125,0.100000,0.111111,0.700000,0.636364,0.666667,0.333333,0.272727,0.300000,0.125,0.100000,0.111111
2,0.500000,0.714286,0.588235,0.333333,0.500000,0.600000,0.125,0.200000,0.153846,0.500000,0.714286,0.588235,0.333333,0.500000,0.400000,0.125,0.200000,0.153846
3,0.500000,0.625000,0.555556,0.333333,0.428571,0.562500,0.250,0.333333,0.285714,0.600000,0.750000,0.666667,0.333333,0.428571,0.375000,0.250,0.333333,0.285714
4,0.500000,0.555556,0.526316,0.333333,0.375000,0.529412,0.125,0.142857,0.133333,0.500000,0.555556,0.526316,0.333333,0.375000,0.352941,0.125,0.142857,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11525,0.500000,0.214286,0.300000,0.200000,0.076923,0.277778,0.000,0.000000,0.000000,0.500000,0.214286,0.300000,0.200000,0.076923,0.111111,0.000,0.000000,0.000000
11526,0.500000,0.333333,0.400000,0.200000,0.125000,0.384615,0.000,0.000000,0.000000,0.500000,0.333333,0.400000,0.200000,0.125000,0.153846,0.000,0.000000,0.000000
11527,0.333333,0.200000,0.250000,0.200000,0.111111,0.238095,0.000,0.000000,0.000000,0.333333,0.200000,0.250000,0.200000,0.111111,0.142857,0.000,0.000000,0.000000
11528,0.333333,0.333333,0.333333,0.200000,0.200000,0.333333,0.000,0.000000,0.000000,0.333333,0.333333,0.333333,0.200000,0.200000,0.200000,0.000,0.000000,0.000000


In [59]:
df.columns

Index(['precision1gram', 'recall1gram', 'f1gram', 'precision2gram',
       'recall2gram', 'f2gram', 'precision3gram', 'recall3gram', 'f3gram',
       'precision1stem', 'recall1stem', 'f1stem', 'precision2stem',
       'recall2stem', 'f2stem', 'precision3stem', 'recall3stem', 'f3stem'],
      dtype='object')

In [71]:
import numpy as np
train_X = df.to_numpy()
train_y = np.array(train_gt)
print(train_X.shape)
print(train_y.shape)

(11530, 18)
(11530,)


# Fitting on Linear Models

In [72]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_X, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [77]:
# preparing the test/pred set
from sklearn.preprocessing import StandardScaler

# preparing data points
test_gts = []
sent1_data, sent2_data = [], []
test_features = []
with open(dev_dp, 'r') as f:
    lines = f.readlines() #[choose:choose+1]
    
    for line in tqdm(lines, total=len(lines)):
        comp = line.split('\t')
        # gt = get_gt_label(comp[4])
        
        # print(comp[4][1:-1].split(','), comp[4])
        gt = get_gt_train_label(comp[4][1:-1].split(','))
        if gt == -1:
            continue
        sent1_data.append(comp[2])
        sent2_data.append(comp[3])
        test_gts.append(gt)
        
        feature = paraphrase_Das_features(comp[2], comp[3])
        test_features.append(feature)
        
test_df = pd.DataFrame.from_records(test_features)
test_X = test_df.to_numpy()
test_y = np.array(test_gts)

preds = clf.predict(test_X)
print(test_X.shape, test_y.shape, preds.shape)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4727/4727 [00:02<00:00, 1612.88it/s]


(4142, 18) (4142,) (4142,)


In [78]:
def compute_precision(y_true, y_pred, pos_class=True):
    true_neg, false_pos, false_neg, true_pos = confusion_matrix(y_true, y_pred).ravel()
    precision = 0.0
    if pos_class:
        if true_pos+false_pos != 0:
            precision = true_pos/(true_pos+false_pos)
    else:
        if true_neg+false_neg != 0:
            precision = true_neg/(true_neg+false_neg)
    return precision

def compute_recall(y_true, y_pred, pos_class=True):
    true_neg, false_pos, false_neg, true_pos = confusion_matrix(y_true, y_pred).ravel()
    recall = 0.0
    if pos_class:
        if true_pos+false_neg != 0:
            recall = true_pos/(true_pos+false_neg)
    else:
        if true_neg+false_pos!=0:
            recall = true_neg/(true_neg+false_pos)
    return recall

def compute_fscore(y_true, y_pred, pos_class = True, beta=1):
    precision = compute_precision(y_true, y_pred, pos_class)
    recall = compute_recall(y_true, y_pred, pos_class)
    fscore = 0.0
    if precision+recall!=0:
        fscore = (1+math.pow(beta,2))*(precision*recall)/(math.pow(beta,2)*precision + recall)
    return fscore

In [79]:
# scores for positive class
print(f'precision: {compute_precision(test_gts, preds)}, recall: {compute_recall(test_gts, preds)}, f1: {compute_fscore(test_gts, preds)}')

precision: 0.7774607703281027, recall: 0.3707482993197279, f1: 0.5020727775218793


In [80]:
# scores for negative class
print(f'precision: {compute_precision(test_gts, preds, pos_class=False)}, recall: {compute_recall(test_gts, preds, pos_class=False)}, f1: {compute_fscore(test_gts, preds, pos_class=False)}')

precision: 0.7311827956989247, recall: 0.9416167664670658, f1: 0.8231637493865532
